In [1]:
# imports and basic settings

import pandas as pd
import numpy as np
import scipy.stats as st
import statistics
import ipywidgets as widgets
from IPython.display import display, clear_output
from pandas.io.formats import style
from IPython.core.display import display, HTML
display(HTML("<style>div.output_scroll { height: 90em; }</style>"))

In [2]:
# wczytanie danych

df = pd.read_csv("IBM_HR.csv")
df["Satisfaction"] = (df['JobSatisfaction'] + df['EnvironmentSatisfaction'] + df['RelationshipSatisfaction'])/3
df

,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,...,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager,Satisfaction
0,41,Yes,Travel_Rarely,1102,Sales,1,2,Life Sciences,1,1,...,80,0,8,0,1,6,4,0,5,2.333333
1,49,No,Travel_Frequently,279,Research & Development,8,1,Life Sciences,1,2,...,80,1,10,3,3,10,7,1,7,3.000000
2,37,Yes,Travel_Rarely,1373,Research & Development,2,2,Other,1,4,...,80,0,7,3,3,0,0,0,0,3.000000
3,33,No,Travel_Frequently,1392,Research & Development,3,4,Life Sciences,1,5,...,80,0,8,3,3,8,7,3,0,3.333333
4,27,No,Travel_Rarely,591,Research & Development,2,1,Medical,1,7,...,80,1,6,3,3,2,2,2,2,2.333333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1465,36,No,Travel_Frequently,884,Research & Development,23,2,Medical,1,2061,...,80,1,17,3,3,5,2,0,3,3.333333
1466,39,No,Travel_Rarely,613,Research & Development,6,1,Medical,1,2062,...,80,1,9,5,3,7,7,1,7,2.000000
1467,27,No,Travel_Rarely,155,Research & Development,4,3,Life Sciences,1,2064,...,80,1,6,0,3,6,2,0,3,2.000000
1468,49,No,Travel_Frequently,1023,Sales,2,3,Medical,1,2065,...,80,0,17,3,2,9,6,0,8,3.333333


In [3]:
#widgets
style = {'description_width': 'initial'}

# department
widget_department = widgets.Select(
    options= sorted(df['Department'].unique().tolist()),
    description='Department',
    style= style,
    disabled=False,
    layout={'width': 'max-content'})


def update_job_role_handler(option_choice):
    widget_job_role_options = sorted(df[df['Department'] == widget_department.value]['JobRole'].unique().tolist())
    widget_job_role.options = widget_job_role_options 

# job role
widget_job_role = widgets.Select(
    options= sorted(df['JobRole'].unique().tolist()),
    description='Job role',
    style=style,
    disabled=False,
    layout={'width': 'max-content'}
)
def update_job_level_handler(option_choice):
    widget_job_level_options = sorted(df[df['JobRole'] == widget_job_role.value]['JobLevel'].unique().tolist())
    widget_job_level.options = widget_job_level_options

widget_job_level = widgets.RadioButtons(
    options = sorted(df[df['JobRole'] == widget_job_role.value]['JobLevel'].unique().tolist()),
    description='Job level',
    style=style,
    disabled=False,
    layout={'width': 'max-content'}
)
widgets.interactive_output(update_job_level_handler, {'option_choice': widget_job_role})
widgets.interactive_output(update_job_role_handler, {'option_choice': widget_department})

#  Gender
widget_gender = widgets.RadioButtons(
    options=['Female', 'Male', 'I prefer not to say'],
    value='Female',
    description='Gender',
    style=style,
    disabled=False,
    layout={'width': 'max-content'})

# Education Field
widget_education_field = widgets.Select(
    options=['Human Resources', 'Life Sciences', 'Marketing', 'Medical', 'Technical Degree', 'Other'],
    value='Human Resources',
    description='Education Field',
    style=style,
    disabled=False,
    layout={'width': 'max-content'})

# Education Level
widget_education_level = widgets.Select(
    options=[('1 - Below College', 1),('2 - College', 2),('3 - Bachelor', 3), ('4 - Master', 4), ('5 - Doctor', 5)],
    value=1,
    description='Education Level',
    style=style,
    disabled=False,
    layout={'width': 'max-content'})


# Salary 
widget_monthly_income = widgets.IntSlider(
    min= df['MonthlyIncome'].min(),
    max = df['MonthlyIncome'].max(), 
    description='Monthly Income',
    style = style)
# Total Working Years
widget_total_years = widgets.IntSlider(
    min= df['TotalWorkingYears'].min(),
    max = df['TotalWorkingYears'].max(),
    description='Total Working Years',
    style = style)
#department change?
widget_department_change = widgets.RadioButtons(
    options=['Yes','No'],
    value = 'Yes',
    style=style,
    description='Do you want to change your department?')

#role change?

def update_role_change(*args):
    if widget_department_change.value == 'Yes':
        widget_role_change.disabled = True
        widget_role_change.value = 'Yes'
    else:
        widget_role_change.disabled = False
widget_department_change.observe(update_role_change,'value')

widget_role_change = widgets.RadioButtons(
    options=['Yes', 'No'],
    value = 'Yes',
#    value='pineapple', # Defaults to 'pineapple'
#    layout={'width': 'max-content'}, # If the items' names are long
    style=style,
    disabled = True,
    description='Do you want to change your role?')

#chosing goal

widget_goal = widgets.RadioButtons(
    options=['Salary', 'Promotion'],
#    value='pineapple', # Defaults to 'pineapple'
#    layout={'width': 'max-content'}, # If the items' names are long
    style=style,
    disabled = False,
    description='Recomandation goal:')

def update_goal(*args):
    if widget_department_change.value  == 'Yes' or widget_role_change.value == 'Yes':
        if widget_job_level.value == 5:
            widget_goal.disabled = True
            widget_goal.value = 'Salary'
        else:
             widget_goal.disabled = False
    else:
        if widget_job_level.value == max(widget_job_level.options):
            widget_goal.disabled = True
            widget_goal.value = 'Salary'
        else:
            widget_goal.disabled = False
widget_department_change.observe(update_goal,'value')
widget_role_change.observe(update_goal,'value')
widget_job_level.observe(update_goal,'value')

#Training last year
widget_training = widgets.IntSlider(
    min = df['TrainingTimesLastYear'].min(),
    max = df['TrainingTimesLastYear'].max(),
    description='Training Times Last Year',
    style = style)
display(widget_training)


# widget handler
values = {'department_dropdown': widget_department.value,
          'education_level_dropdown': widget_education_level.value,
          'education_field_dropdown': widget_education_field.value,
          'job_role_dropdown': widget_job_role.value,
          'job_level_dropdown': widget_job_level.value,
          'gender_dropdown' : widget_gender.value,
          'total_years_slider' : widget_total_years.value,
          'monthly_income_slider' : widget_monthly_income.value,
          'goal_buttons': widget_goal.value,
          'department_change_buttons':widget_department_change.value,
          'role_change_buttons':widget_role_change.value,
          'training_slider' : widget_training.value}
def widgets_handler(department_change_buttons_val,
                    role_change_buttons_val,
                    department_dropdown_val,
                    education_level_dropdown_val,
                    education_field_dropdown_val,
                    job_role_dropdown_val,
                    job_level_dropdown_val,
                    gender_dropdown_val,
                    total_years_slider_val,
                    monthly_income_slider_val,
                    goal_buttons_val,
                    training_slider_val):
    values['department_change_buttons'] =  department_change_buttons_val
    values['role_change_buttons'] =  role_change_buttons_val
    values['department_dropdown'] = department_dropdown_val
    values['education_level_dropdown'] = education_level_dropdown_val
    values['education_field_dropdown'] = education_field_dropdown_val
    values['job_role_dropdown'] = job_role_dropdown_val
    values['job_level_dropdown'] = job_level_dropdown_val
    values['gender_dropdown'] = gender_dropdown_val
    values['total_years_slider'] =  total_years_slider_val
    values['monthly_income_slider'] =  monthly_income_slider_val
    values['goal_buttons'] =  goal_buttons_val
    values['training_slider'] = training_slider_val





IntSlider(value=0, description='Training Times Last Year', max=6, style=SliderStyle(description_width='initial…

In [4]:
########################################################################################
#filtrowanie df z wykorzystaniem słownika values utworzonego z widgetów"""

def df_filter_recommendation(department_change_buttons_val,
                             role_change_buttons_val,
                             department_dropdown_val,
                             education_level_dropdown_val,
                             education_field_dropdown_val,
                             job_role_dropdown_val,
                             total_years_slider_val,
                             goal_buttons_val,
                             training_slider_val,
                             monthly_income_slider_val = widget_monthly_income.value,
                             job_level_dropdown_val=widget_job_level.value):

    # filtrowanie goal
    if widget_goal.value == "Salary":
        filtr_goal = df['MonthlyIncome']> (1.1 * monthly_income_slider_val)
    else:
        filtr_goal = df['JobLevel']>widget_job_level.value
        
    if widget_department_change.value == 'No':
        filtr_dep = df["Department"] == widget_department.value
    else:
        filtr_dep = df["Department"].isin(df["Department"].unique())

    if widget_role_change.value == 'No':
        filtr_role = df["JobRole"] == widget_job_role.value
    else:
        filtr_role = df["JobRole"].isin(df["JobRole"].unique())
   
    
    df_filtred_recom = df[(filtr_dep)  &
               (filtr_role)  &
               (filtr_goal)]
    return df_filtred_recom
filtr_department = df["Department"] == widget_department.value
filtr_job_role = df["JobRole"] == widget_job_role.value



def continious_output(department_dropdown_val=widget_department,
                      job_role_dropdown_val=widget_job_role,
                      job_level_dropdown_val=widget_job_level,
                      department_change_buttons_val = widget_department_change,
                      role_change_buttons_val = widget_role_change,
                      education_level_dropdown_val=widget_education_level,
                      education_field_dropdown_val=widget_education_field,
                      total_years_slider_val = widget_total_years,
                      monthly_income_slider_val = widget_monthly_income,
                      training_slider_val = widget_training,
                      goal_buttons_val = widget_goal):
    df_filtred = df_filter_recommendation(department_change_buttons_val,
                                          role_change_buttons_val = widget_role_change.value,
                                          department_dropdown_val = widget_department.value,
                                          education_level_dropdown_val = widget_education_level.value,
                                          education_field_dropdown_val = widget_education_field.value,
                                          job_role_dropdown_val = widget_job_role.value,
                                          total_years_slider_val = widget_total_years.value,
                                          goal_buttons_val = widget_goal.value,
                                          training_slider_val = widget_training.value,
                                          monthly_income_slider_val = widget_monthly_income.value,
                                          job_level_dropdown_val=widget_job_level.value)
    years=df_filtred['TotalWorkingYears']
    if len(years)>0:
        years = round(years.mean())
    else:
        years = 'n/a'
    education = df_filtred['Education']
    if len(education)>0:
        education_mode=statistics.mode(education)
        education = round(education.mean())
    else:
        education = 'n/a'
        education_mode = 'n/a'
    training = df_filtred['TrainingTimesLastYear']
    if len(training)>0:
        training = round(training.mean())
    else:
        training = 'n/a'
    grouped_data = df_filtred[(df_filtred['Education']<= widget_education_level.value) & 
                              (df_filtred['EducationField'] == widget_education_field.value) & 
                              (df_filtred['TrainingTimesLastYear'] <= widget_training.value) &  
                              (df_filtred['TotalWorkingYears'] <= widget_total_years.value)].groupby(['Department','JobRole','JobLevel'])['MonthlyIncome'].mean()
    grouped_df = grouped_data.to_frame()
    def print_stats():
        grouped_df_for_stats = df_filtred[df_filtred['EducationField'] == widget_education_field.value
                                         ].groupby(['Department',
                                                    'JobRole',
                                                     'JobLevel'])[['MonthlyIncome',
                                                                   'Education',
                                                                   'EducationField',
                                                                   'TrainingTimesLastYear',
                                                                   'TotalWorkingYears']].mean().round(2)
        if len(grouped_df_for_stats)==0:
            print('There is no employees with chosen data matching your criteria in our database')
        else:
            display(grouped_df_for_stats)
            
    if widget_goal.value == "Salary":
        print('You earn more than:')
        print(f'\033[1m{round(st.percentileofscore(df["MonthlyIncome"], widget_monthly_income.value))}% in your company\033[0m')
        print(f'\033[1m{round(st.percentileofscore(df[filtr_department]["MonthlyIncome"], widget_monthly_income.value))}% in your department\033[0m')
        print(f'\033[1m{round(st.percentileofscore(df[filtr_job_role]["MonthlyIncome"], widget_monthly_income.value))}% at your position\033[0m')
        print('')
        print(f'There is \033[1m{len(df_filtred.index)}\033[0m people earning at least 10% more than you in chosen company/department/role')
        print('')
        print(f'Your working years is \033[1m{widget_total_years.value}\033[0m and average working years of people earning more than you is \033[1m{(years)}\033[0m')
        print('')
        print(f'Your education level is \033[1m{widget_education_level.value}\033[0m and dominant value of education level of people earning more than you is \033[1m{education_mode}\033[0m')
        print('')
        print(f'You had \033[1m{widget_training.value}\033[0m times training last year and people earning more than you had in average \033[1m{(training)}\033[0m training last year')
        if (widget_department_change.value == 'Yes') and (widget_role_change.value == 'Yes') and len(grouped_df)>0:
            print('\n\033[1mList of positions and levels of employees with similar required education, training time, total working years\nin your company earning in average more than you:\033[0m')
            display(grouped_data[grouped_df['MonthlyIncome']> (1.1*monthly_income_slider_val)].to_frame().style.highlight_max(color='lightgreen'))
        elif (widget_department_change.value == 'No') and (widget_role_change.value == 'Yes') and len(grouped_df)>0:
            print('\n\033[1mList of positions and levels of employees with similar required education, training time, total working years\nin your department earning in average more than you:\033[0m')
            display(grouped_data.to_frame().style.highlight_max(color='lightgreen'))
        elif (widget_department_change.value == 'No') and (widget_role_change.value == 'No') and len(grouped_df)>0:
            print('\n\033[1mList of levels of employees with similar required education, training time, total working years\nearning in average more than you on the same job role:\033[0m')
            display(grouped_df.style.highlight_max(color='lightgreen'))
        elif (widget_department_change.value == 'Yes') and (widget_role_change.value == 'Yes') and len(grouped_df)==0:
            print('\n\033[1mAverage statistics of people with the same education field earning more than you in whole company:\033[0m')
            print_stats()
        elif (widget_department_change.value == 'No') and (widget_role_change.value == 'Yes') and len(grouped_df)==0:
            print('\n\033[1mAverage statistics of people with the same education field earning more than you in your department:\033[0m')
            print_stats()
        else:
            print('\n\033[1mAverage statistics of people with the same education field earning more than you on your position:\033[0m')
            print_stats()
    if widget_goal.value == "Promotion":
        print(f'You are at higher level than:')
        print(f'\033[1m{round(st.percentileofscore(df["JobLevel"], widget_job_level.value))}% of employees in your company\033[0m')
        print(f'\033[1m{round(st.percentileofscore(df[filtr_department]["JobLevel"], widget_job_level.value))}% in your department\033[0m')
        print(f'\033[1m{round(st.percentileofscore(df[filtr_job_role]["JobLevel"], widget_job_level.value))}% at your position\033[0m')
        print('')
        print(f'There is \033[1m{len(df_filtred.index)}\033[0m people on higher position in chosen company/department/role')
        print('')
        print(f'Your working years is \033[1m{widget_total_years.value}\033[0m and average working years of people at higher level than you is \033[1m{(years)}\033[0m')
        print('')
        print(f'Your education level is \033[1m{widget_education_level.value}\033[0m and dominant value of education level of people at higher level than you is \033[1m{education_mode}\033[0m')
        print('')
        print(f'You had \033[1m{widget_training.value}\033[0m times training last year and people at higher position than you had in average \033[1m{(training)}\033[0m training last year')
        if (widget_department_change.value == 'Yes') and (widget_role_change.value == 'Yes') and len(grouped_df)>0:
            print('\n\033[1mList of job roles in your company with similar required education, training time, total working years,\nbut with higher level of job role than you :\033[0m')
            display(grouped_data.to_frame().style.highlight_max(color='lightgreen'))
        elif (widget_department_change.value == 'Yes') and (widget_role_change.value == 'Yes') and len(grouped_df)==0:
            print('\n\033[1mAverage statistics of people in whole company with higher position, but with same education field:\033[0m')
            print_stats()  
        elif (widget_department_change.value == 'No') and (widget_role_change.value == 'Yes') and len(grouped_df)>0:
            print('\n\033[1mList of job roles in your department with similar required education, training time, total working years,\nbut with higher level of job role than you:\033[0m')
            display(grouped_data.to_frame().style.highlight_max(color='lightgreen'))
        elif (widget_department_change.value == 'No') and (widget_role_change.value == 'Yes') and len(grouped_df)==0:
            print('\n\033[1mAverage statistics of people in your department with higher position, but with same education field:\033[0m')
            print_stats() 
        else:
            print('\n\033[1mAverage statistics of people at your position with higher position level, but with same education field:\033[0m')
            print_stats()        


In [5]:
widget = widgets.interactive(continious_output)
output = widget.children[-1]
layout = widgets.Layout(width='auto', height='auto', grid_gap='15px')
controls1 = widgets.HBox(widget.children[0:3], layout=layout)
controls2 = widgets.HBox(widget.children[3:5], layout=layout)
controls3 = widgets.HBox(widget.children[5:7], layout=layout)
controls4 = widgets.HBox(widget.children[7:10], layout=layout)
controls5 = widgets.HBox(widget.children[10:-1], layout=layout)

In [6]:
print('\033[1mRecommendations\033[0m')
display(widgets.VBox([controls1,
                      controls2,
                      controls3,
                      controls4,
                      controls5,
                      output], layout=widgets.Layout(margin = '25px 0px 25px 0px',grid_gap='30px')))


Recommendations
